In [76]:
from eva import *
from eva.ckks import *
from eva.seal import *
import cv2
import numpy as np
import math

In [77]:
def rgbToGray(b,g,r):
    bv = [0.07]*4096
    bg = [0.72]*4096
    br = [0.21]*4096
    
    return b*0.07 + g*0.72 + r*0.21

In [78]:
def closest(lst, K):
      
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]

In [79]:
def getImg(imgpath):
    li = list(map(lambda x: 2 ** x, range(9)))
    img = cv2.imread(imgpath)
    row,col,ch = img.shape
    vec_size = row*col
    resized = img
    if row>256 or col>256:
        print("Max")
        dim = (256, 256)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        vec_size = 256*256
        return resized, vec_size
        
    if row not in li:
        clo = closest(li, row)
        dim = (clo, clo)
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        vec_size = col*col
        return resized, vec_size
        
        
    if row != col:
        if row>col:
            dim = (row, row)
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            vec_size = row*row
            return resized, vec_size
            
        else:
            dim = (col, col)
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            vec_size = col*col
            return resized, vec_size
            
            
    return resized, vec_size
            
            
    

In [80]:
img, vec_size = getImg('images/john-mark-smith-NslBwDGhCf0-unsplash.jpg')
vec_size

Max


65536

In [81]:
poly = EvaProgram('Polynomial', vec_size)
with poly:
    b = Input('b')
    g = Input('g')
    r = Input('r')
    Output('y', rgbToGray(b,g,r))

In [82]:
poly.set_output_ranges(30)
poly.set_input_scales(30)
compiler = CKKSCompiler()
compiled_poly, params, signature = compiler.compile(poly)
public_ctx, secret_ctx = generate_keys(params)

In [83]:
#img = cv2.imread('images/baboon.png')


In [84]:
lib= img[:,:,0].flatten()
lig= img[:,:,1].flatten()
lir= img[:,:,2].flatten()

In [85]:
inputs = { 'b':lib, 'g':lig, 'r':lir}
encInputs = public_ctx.encrypt(inputs, signature)

In [86]:
encOutputs = public_ctx.execute(compiled_poly, encInputs)

In [87]:
outputs = secret_ctx.decrypt(encOutputs, signature)

In [88]:
size = int(math.sqrt(vec_size))
gr = np.array(outputs.get('y'))
cv2.imwrite("test1.png" , gr.reshape(size,size)) 

True